# Preparing Data

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
base_dir= "/content/drive/My Drive/Cal State LA/Machine Learning/Datasets/Datasets-v2/"
laptops = "laptops_cleaned.json"
#meta_ranges = "meta_Ranges_Ovens_Cooktops.json"
#meta_laundry = "meta_Laundry_Appliancess.json"
#apl = "/content/drive/My Drive/Cal State LA/Machine Learning/Datasets/Appliances.json"

In [3]:
df_laptops=pd.read_json(base_dir+laptops)
#df_apl = pd.read_json(apl, lines=True)
#df_laundry_joined = pd.merge(df_meta_laundry, df_apl, how='inner', on="asin")
#df_laundry_joined = df_laundry_joined.drop(columns=['image_y', 'date', 'feature','unixReviewTime', 'summary','reviewerName','style','reviewerID','reviewTime','verified','C1', 'title', 'also_buy', 'tech2', 'category_len','brand','rank','also_view','similar_item','details','main_cat','tech1','image_x'])

In [4]:
df_laptops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60367 entries, 0 to 60366
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   overall      60367 non-null  int64  
 1   vote         60367 non-null  int64  
 2   verified     60367 non-null  int64  
 3   asin         60367 non-null  object 
 4   reviewText   60367 non-null  object 
 5   summary      60362 non-null  object 
 6   category     60367 non-null  object 
 7   description  60367 non-null  object 
 8   title        60367 non-null  object 
 9   rank         60367 non-null  object 
 10  price        60367 non-null  float64
 11  rank_length  60367 non-null  int64  
 12  rank_1       60367 non-null  int64  
 13  rank_2       60367 non-null  int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 6.9+ MB


In [5]:
df_laptops.head()

,overall,vote,verified,asin,reviewText,summary,category,description,title,rank,price,rank_length,rank_1,rank_2
0,5,3,0,B00005NBIS,you can't have everything at least not at 1199...,The perfect compromise,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
1,5,3,0,B00005NBIS,you can't have everything at least not at 1199...,The perfect compromise,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
2,5,2,0,B00005NBIS,i bought the one with a 20 gb hard drive and i...,Basic 500 MHz iBook,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
3,5,2,0,B00005NBIS,i bought the one with a 20 gb hard drive and i...,Basic 500 MHz iBook,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
4,5,34,0,B00005NBIS,apple has reinvented the ibook as one of the b...,A great laptop! *****,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593


In [6]:
len(pd.unique(df_laptops['asin']))

2584

# Loading LDA Models

In [7]:
import gensim
from gensim import corpora, models

In [8]:
lda_model_laptop_review = gensim.models.ldamodel.LdaModel.load(base_dir+'lda_model_laptop_review_15.model')  


In [9]:
from pprint import pprint
 
# Print the Keyword in the 20 topics
pprint(lda_model_laptop_review.print_topics())

[(0,
  '0.052*"key" + 0.051*"keyboard" + 0.037*"use" + 0.025*"touch" + 0.022*"pad" '
  '+ 0.021*"button" + 0.021*"mouse" + 0.019*"right" + 0.017*"screen" + '
  '0.014*"click"'),
 (1,
  '0.077*"usb" + 0.073*"port" + 0.038*"device" + 0.037*"wifi" + 0.030*"power" '
  '+ 0.029*"connect" + 0.027*"wireless" + 0.025*"connection" + '
  '0.022*"bluetooth" + 0.018*"hdmi"'),
 (2,
  '0.103*"tablet" + 0.096*"great" + 0.095*"good" + 0.055*"work" + '
  '0.051*"price" + 0.050*"product" + 0.047*"excellent" + 0.019*"far" + '
  '0.018*"value" + 0.018*"fast"'),
 (3,
  '0.054*"buy" + 0.037*"love" + 0.031*"use" + 0.030*"year" + 0.022*"get" + '
  '0.019*"great" + 0.018*"old" + 0.018*"good" + 0.017*"purchase" + '
  '0.015*"new"'),
 (4,
  '0.029*"game" + 0.019*"high" + 0.018*"play" + 0.016*"graphic" + '
  '0.016*"performance" + 0.015*"run" + 0.014*"well" + 0.013*"price" + '
  '0.013*"processor" + 0.013*"gaming"'),
 (5,
  '0.063*"screen" + 0.035*"quality" + 0.029*"feel" + 0.024*"build" + '
  '0.020*"keyboard" +

In [10]:
#lda_model_laptop_des = gensim.models.ldamodel.LdaModel.load(base_dir+'lda_model_laptop_des.model')  

In [11]:
#from pprint import pprint
 
# Print the Keyword in the 20 topics
#pprint(lda_model_laptop_des.print_topics())

In [12]:
with open(base_dir+'laptop_review_corpus.pkl','rb') as f:
  corpus=pickle.load(f)

In [13]:
#with open(base_dir+'laptop_des_corpus.pkl','rb') as f:
#  corpus_des=pickle.load(f)

# Creating Topic Vectors

In [14]:
review_topics = []
des_topics =[]
vote = []
topic_vec=[]
util_rev = []
hedo_rev = []
amb_rev = []
util_des = []
hedo_des = []
amb_des = []
for i in range(len(df_laptops)):
    top_topics_review = lda_model_laptop_review.get_document_topics(corpus[i], minimum_probability=0.0)
    #top_topics_des = lda_model_laptop_des.get_document_topics(corpus_des[i], minimum_probability=0.0)
    topic_vec_review = [top_topics_review[i][1] for i in range(15)]
    topic_vec_review_sum = sum(topic_vec_review)
    #topic_vec_des = [top_topics_des[i][1] for i in range(8)]
    #topic_vec_des_sum = sum(topic_vec_des)
    util_rev.append((topic_vec_review[1]+topic_vec_review[6]+topic_vec_review[9]+topic_vec_review[11]+topic_vec_review[14])/topic_vec_review_sum)
    hedo_rev.append((topic_vec_review[3]+topic_vec_review[4]+topic_vec_review[5]+topic_vec_review[10]+topic_vec_review[12])/topic_vec_review_sum)
    amb_rev.append((topic_vec_review[0]+topic_vec_review[2]+topic_vec_review[7]+topic_vec_review[8]+topic_vec_review[13])/topic_vec_review_sum)
    #util_des.append((topic_vec_des[1]+topic_vec_des[3]+topic_vec_des[6]+topic_vec_des[7])/topic_vec_des_sum)
    #hedo_des.append((topic_vec_des[0]+topic_vec_des[4])/topic_vec_des_sum)
    #amb_des.append((topic_vec_des[2]+topic_vec_des[5])/topic_vec_des_sum)
    #topic_vec.extend([df_inner.iloc[i].helpful])
    review_topics.append(topic_vec_review)
    #des_topics.append(topic_vec_des)

Topic vector for the last review

In [15]:
topic_vec_review

[0.0026666685,
 0.082666665,
 0.0026666685,
 0.0026666704,
 0.11420219,
 0.002666671,
 0.0026666673,
 0.0026666669,
 0.0026666706,
 0.0026666715,
 0.77113116,
 0.0026666692,
 0.0026666855,
 0.0026666669,
 0.0026666685]

In [16]:
df_laptops['reviewText'][60366]

'overall this is a nice large desktop replacement laptop it has a nice screen keyboard feels nice 2 usb 30 ports bluetooth and a cddvd drive coupled with an intel i3 processor its nice and snappy too i would recommend this computer to anyone looking for a nice big laptop that pretty much has all a basic consumer could want'

# Finding the index of reviews with highest % of hedonic, utilitarian, or mixed topics

In [17]:
max_util = pd.Series(util_rev).idxmax()

In [18]:
max_hedo= pd.Series(hedo_rev).idxmax()

In [19]:
max_amb = pd.Series(amb_rev).idxmax()

In [20]:
df_laptops['reviewText'][max_util]

"the dell xps laptop is the worst computer i've ever had do not buy it -- it's a waste of money less than one year of owning it & it freezes at least 1xevery day the bsod of death also shows up there answer was to send it in to their depost and to replace the hardware and reinstall the operating system all of my files were lost and i had to purchase programs again everything from microsoft office to photoshop less than 2 weeks after having the laptop back & reinstalling everything it has started freezing again dell says that since they fixed it once they say the case is closed worst customer service i used to be a faithful dell customer but now i'm gone"

In [21]:
df_laptops['reviewText'][max_hedo]

'this laptop feels so premium in my opinion i prefer this over the macbook air because it feels premium and it looks premium this screen is really good for 200$ less than the macbook air it is so worth the money because it performs better than the macbook air it feels like premium metal externally this laptop for being 133 inch is not that small its the perfect size when it comes to portablility its also blazing fast for a intel core i5 8mb ram processor you will have a better experience with this laptop over the macbook air or any ultrabook in my opinion'

In [22]:
df_laptops['reviewText'][max_amb]

'works perfect for me  yes the mouse pad is annoying but i attached a wireless mouse for my own convenience  oh and the headphone jack is in the back which seems like a stupid design idea  but works fine and i am happy with my purchase'

# Combining into New DataFrame

In [48]:
regression_df= pd.DataFrame()
regression_df['asin'] = df_laptops['asin']
regression_df['vote'] = df_laptops['vote']
regression_df['hedo'] = hedo_rev
regression_df['util'] = util_rev
regression_df['overall'] = df_laptops['overall']
regression_df['verified'] = df_laptops['verified']
regression_df['rank_2'] = df_laptops['rank_2']
regression_df['review_length'] = df_laptops.reviewText.str.len()

In [49]:
regression_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60367 entries, 0 to 60366
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   asin           60367 non-null  object 
 1   vote           60367 non-null  int64  
 2   hedo           60367 non-null  float64
 3   util           60367 non-null  float64
 4   overall        60367 non-null  int64  
 5   verified       60367 non-null  int64  
 6   rank_2         60367 non-null  int64  
 7   review_length  60367 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 6.6+ MB


In [50]:
with open(base_dir+'regression_df.pkl', 'wb') as f:
  pickle.dump(regression_df, f)

# Pickle Load

In [52]:
with open('/content/drive/My Drive/Cal State LA/Machine Learning/Datasets/Datasets-v2/regression_df.pkl', 'rb') as f:
  regression_df = pickle.load(f)

# OLS Regression Model

In [24]:
#diff_hedo = np.subtract(hedo_des, hedo_rev)
#diff_util = np.subtract(util_des, util_rev)
#diff_amb = np.subtract(amb_des, amb_rev)

In [59]:
import numpy as np
import statsmodels.api as sm # import statsmodels 

x1 = np.array(regression_df[['hedo']])
x2 = np.array(regression_df[['util']])
x3= np.array(regression_df[['overall']])
x4= np.array(regression_df[['verified']])
x5= np.array(regression_df[['rank_2']])
x6= np.array(regression_df[['review_length']])
#x6 = hedo_rev*df_laptops['review_length']
#x7 = util_rev*df_laptops['review_length']


X = np.column_stack((x1, x2, x3, x4, x5))
X = sm.add_constant(X)
y = np.array((regression_df.vote.astype(int)))

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
#model2 = sm.OLS(y1, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)
# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     67.87
Date:                Tue, 22 Dec 2020   Prob (F-statistic):           5.50e-71
Time:                        02:30:47   Log-Likelihood:            -3.1395e+05
No. Observations:               60367   AIC:                         6.279e+05
Df Residuals:                   60361   BIC:                         6.280e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.6645      1.032      7.429      0.000       5.642       9.686
x1            -0.5307      0.930     -0.571      0.568      -2.353       1.292
x2             3.2426      1.152      2.814      0.005       0.984       5.501
x3             0.8839      0.152      5.797      0.000       0.585       1.183
x4            -7.8367      0.442    -17.725      0.000      -8.703      -6.970
x5         -7.132e-05    2.2e-05     -3.239      0.001      -0.000   -2.82e-05
==============================================================================
Omnibus:                   218526.175   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     246967422307.184
Skew:                          74.836   Prob(JB):                         0.00
Kurtosis:                    9910.774   Cond. No.                     9.04e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Poisson Regression

In [26]:
import statsmodels.api as sm
from statsmodels.formula.api import poisson
from statsmodels.formula.api import negativebinomial

In [54]:
x1 = np.array(regression_df[['hedo']])
x2 = np.array(regression_df[['util']])
x3= np.array(regression_df[['overall']])
x4= np.array(regression_df[['verified']])
x5= np.array(regression_df[['rank_2']])
x6= np.array(regression_df[['review_length']])
#x6 = hedo_rev*df_laptops['review_length']
#x7 = util_rev*df_laptops['review_length']


X = np.column_stack((x1, x2, x3, x4, x5))
X = sm.add_constant(X)
y = np.array((regression_df.vote.astype(int)))

poisson_model = sm.Poisson(y, X).fit(maxiter=500)
poisson_model.summary()

Optimization terminated successfully.
         Current function value: 12.949027
         Iterations 95


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                60367
Model:                        Poisson   Df Residuals:                    60361
Method:                           MLE   Df Model:                            5
Date:                Mon, 21 Dec 2020   Pseudo R-squ.:                 0.06255
Time:                        22:02:29   Log-Likelihood:            -7.8169e+05
converged:                       True   LL-Null:                   -8.3386e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7296      0.010    164.792      0.000       1.709       1.750
x1            -0.1325      0.010    -13.261      0.000      -0.152      -0.113
x2             0.6021      0.012     51.174      0.000       0.579       0.625
x3             0.1671      0.002    106.147      0.000       0.164       0.170
x4            -1.1573      0.004   -321.406      0.000      -1.164      -1.150
x5         -1.299e-05   2.31e-07    -56.190      0.000   -1.34e-05   -1.25e-05
==============================================================================
"""

In [56]:
# mixed model 

import statsmodels.formula.api as smf

product_id = np.array(regression_df[['asin']])

md = smf.mixedlm("vote ~ hedo+util+overall+verified+rank_2", regression_df, groups=regression_df['asin'])

mdf = md.fit()

print(mdf.summary())


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmode

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: vote        
No. Observations: 60367   Method:             REML        
No. Groups:       2584    Scale:              1926.4221   
Min. group size:  4       Likelihood:         -313955.4012
Max. group size:  1302    Converged:          No          
Mean group size:  23.4                                    
-----------------------------------------------------------
           Coef.   Std.Err.     z     P>|z|  [0.025  0.975]
-----------------------------------------------------------
Intercept   7.660     1.032    7.422  0.000   5.637   9.683
hedo       -0.534     0.930   -0.574  0.566  -2.357   1.289
util        3.244     1.153    2.815  0.005   0.985   5.503
overall     0.884     0.153    5.798  0.000   0.585   1.183
verified   -7.838     0.442  -17.724  0.000  -8.704  -6.971
rank_2     -0.000     0.000   -3.221  0.001  -0.000  -0.000
Group Var   0.063     0.009                               